In [2]:
!pip install langchain langchain-core langchain-community langchain-openai faiss-cpu pypdf pymupdf tiktoken


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 48.8 MB/s eta 0:00:00


Data Ingestion

In [4]:
from langchain_core.documents import Document

In [6]:
doc=Document(page_content=" this is the main text content I am using to create RAG",
metadata={
    "source":"example.txt",
    "pages":1,
    "author":"suryansh tyagi",
    "date_created":"2025-01-01"
}
)

In [7]:
doc

Document(metadata={'source': 'example.txt', 'pages': 1, 'author': 'suryansh tyagi', 'date_created': '2025-01-01'}, page_content=' this is the main text content I am using to create RAG')

In [8]:
## create a simple txt
import os
os.makedirs("data/text_files",exist_ok=True)

In [11]:
sample_texts={
    "data/text_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",

    "data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems


    """

}

for filepath,content in sample_texts.items():
    with open(filepath,'w',encoding="utf-8") as f:
        f.write(content)

print("✅ Sample text files created!")

✅ Sample text files created!


In [12]:
### TextLoader
from langchain.document_loaders import TextLoader

from langchain_community.document_loaders import TextLoader

loader=TextLoader("data/text_files/python_intro.txt",encoding="utf-8")
document=loader.load()
print(document)

[Document(metadata={'source': 'data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.')]


In [13]:
### Directory Loader
from langchain_community.document_loaders import DirectoryLoader

## load all the text files from the directory
dir_loader=DirectoryLoader(
    "data/text_files",
    glob="**/*.txt", ## Pattern to match files
    loader_cls= TextLoader, ##loader class to use
    loader_kwargs={'encoding': 'utf-8'},
    show_progress=False

)

documents=dir_loader.load()
documents

[Document(metadata={'source': 'data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.'),
 Document(metadata={'source': 'data/text_files/machine_learning.txt'}, page_content='Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Lear

In [17]:
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader

## load all the text files from the directory
dir_loader=DirectoryLoader(
    "data/pdf",
    glob="**/*.pdf", ## Pattern to match files
    loader_cls= PyMuPDFLoader, ##loader class to use
    show_progress=False

)

pdf_documents=dir_loader.load()
pdf_documents

[Document(metadata={'producer': 'iText® 5.5.10 ©2000-2015 iText Group NV (AGPL-version)', 'creator': '', 'creationdate': '2023-05-29T19:50:45+05:30', 'source': 'data/pdf/ece.pdf', 'file_path': 'data/pdf/ece.pdf', 'total_pages': 68, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2023-05-29T19:50:45+05:30', 'trapped': '', 'modDate': "D:20230529195045+05'30'", 'creationDate': "D:20230529195045+05'30'", 'page': 0}, page_content=''),
 Document(metadata={'producer': 'iText® 5.5.10 ©2000-2015 iText Group NV (AGPL-version)', 'creator': '', 'creationdate': '2023-05-29T19:50:45+05:30', 'source': 'data/pdf/ece.pdf', 'file_path': 'data/pdf/ece.pdf', 'total_pages': 68, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2023-05-29T19:50:45+05:30', 'trapped': '', 'modDate': "D:20230529195045+05'30'", 'creationDate': "D:20230529195045+05'30'", 'page': 1}, page_content=''),
 Document(metadata={'producer': 'iText® 5.5.10

In [19]:
type(pdf_documents[0])

langchain_core.documents.base.Document